### First problem: How to determined the value of a variable in a computer program

**Answer: LEGB [local, enclosing, global, builtin]**

*Access global variable inside a scope*

In [1]:
x = "global"

def foo():
    print("x inside:", x)


foo()
print("x outside:", x)

x inside: global
x outside: global


*Modify a global variable inside a scope*

In [2]:
x = "global"

def foo():
    global x  # to be commented
    x += '123'
    print("x inside:", x)


foo()
print("x outside:", x)

x inside: global123
x outside: global123


*Use global variable together with local variable*

In [3]:
x = "global "

def foo():
    global x
    y = "local"
    x = x * 2
    print(x)
    print(y)

foo()

global global 
local


*Use local vairable outside scope*

In [4]:
# def foo():
#     y = "local"


# foo()
# print(y)

<font color='darkred'>**Warning: Avoid Global Variables**</font>                    
Programs with global variables are difficult to maintain and extend because you can no longer view each function as a “black box” that simply receives arguments and returns a result.         

When functions modify global variables, it becomes more difficult to understand the effect of function calls. As programs get larger, this difficulty mounts quickly. Instead of using global variables, use function parameter variables and return values to transfer information from one part of a program to another. 

Global constants, however, are fine. You can place them at the top of a Python source file and access (but not modify) them in any of the functions in the file. Do not use a global declaration to access constants.       

*Nested function - nested scope*

In [5]:
def foo():
    x = 123
    def bar():
        print(x)
    return bar

In [6]:
f = foo()
f()

123


*Memorize nonlocal variable in `.__closure__` when it is* **necessary**

In [7]:
def foo():
    x = 123
    def bar():
        nonlocal x
        x += 1
        print(x)
    return bar

In [8]:
f = foo()

In [9]:
f()
f.__closure__[0].cell_contents

124


124

*Incase nonlocal variable is not nessary for closure function*

In [10]:
def foo():
    x = 123
    def bar():
        print(555)    # you don't have to memroize x inside bar function
    return bar

In [11]:
f = foo()
f()
f.__closure__ is None

555


True

### Nested function & Nested scope

In [13]:
def outer(a):
    b = a                             # b is local variable for outer, a is also a local variable for outer
    def inner():
        c = 3                         # c is local variable of inner, b is non-local variable for inner
        def inner_inner(b):           # new local variable b is defined, b is local for inner_inner, but is not the one defined in outer
            r = b+c                   # r is local variable for inner_inner, c is non-local variable for inner_inner
            return b+c
        return inner_inner
    return inner
foo = outer(10)
bar = foo()

In [203]:
foo.__closure__ is None, bar.__closure__[0].cell_contents

(True, 3)

*Use variable `a` inside `inner`*

In [35]:
def outer(a):
    b = a                            
    def inner():
        c = 3 
        def inner_inner(b):
            r = b+ c + a # use a 
            return b+c
        return inner_inner
    return inner
foo = outer(10)
bar = foo()

In [36]:
foo.__closure__ is None, bar.__closure__[0].cell_contents

(False, 10)

In [37]:
bar(4)

7

*Modify `a` in `inner_inner`*

In [41]:
def outer(a):
    b = a                            
    def inner():
        c = 3 
        def inner_inner(b):
            a += 1
            r = b+ c + a # use a 
            return b+c
        return inner_inner
    return inner
foo = outer(10)
bar = foo()

In [43]:
foo.__closure__ is None, bar.__closure__[0].cell_contents

(True, 3)

In [45]:
# bar(4)

*Although `a` is in outer scope of outer scope of `inner_inner`, it is found in nonlocal space, `__closure__` attribute*

In [46]:
def outer(a):
    b = a                            
    def inner():
        c = 3 
        def inner_inner(b):
            nonlocal a
            a += 1
            r = b + c + a # use a 
            return b+c
        return inner_inner
    return inner
foo = outer(10)
bar = foo()

In [49]:
foo.__closure__ is None, bar.__closure__[0].cell_contents

(False, 11)

In [50]:
bar(4)

7

### Modify nonlocal variable

*Functions are objects, objects have attributes and methods*

In [126]:
def abc():
    pass

setattr(abc,'haha', 'cameldata666')
setattr(abc,'haha_func', lambda x:print(x+1))

print(abc.haha)
abc.haha_func(333)

cameldata666
334


In [114]:
def sample():
    n = 0
    
    # Closure function
    def func():
        print('n=', n)
        
    # Accessor methods for n
    def get_n():
        return n
    def set_n(value):
        nonlocal n
        n = value
        
    # Attach as function attributes
    setattr(func, 'get_n', get_n)
    setattr(func,'set_n', set_n)

    return func

*`n` is defined in factory function `sample`*

In [115]:
sample_func = sample()

In [116]:
sample_func()

n= 0


*Nonlocal variblae `n` modified in `set_n` function*

In [117]:
sample_func.set_n(10)

In [118]:
sample_func()

n= 10


State: game              
Function: with_user_type            

我必须知道你用with_user_type 得state，才能决定你怎么用with_user_type函数                
如果你非要用。。。with_user_type...且被装饰得函数使用得状态还是一直变，那你不如写循环啊    
或者啊，被装饰得函数，要有修改nonlocal variable  - `game`的能力

### Return function, why:
1. Outter function is like a factory, manufacuring functions base on some conditions
2. Inner functions is a specific function to be used directly in business
**The logic of inner function frequently changes overtime, you need to modify code frequently**

**Return / manufacture functions**

In [39]:
# plura exmaple

### A decorator is just a closure

*Only admin get and put food in this store*

In [94]:
class Store(object):
    
    def get_food(self, username):
        if username != 'admin':
            raise Exception("This user is not allowed to get food")
        print('You got it')
    
    def put_food(self, username):
        if username != 'admin':
            raise Exception("This user is not allowed to put food")
        print('You modify it')

In [95]:
s = Store()
s.get_food('admin')

You got it


#### Try think about it

1. Passin a function (`f`) -> return another function (`wrapper`)
2. You get another function (`wrapper`)
3. When you call `wrapper`, `f` is also called
4. Calling `wrapper` complete all functionalities of `f`
5. Calling `wrapper` = Calling `f`
6. No, calling `wrapper` >= `f`


In [96]:
def check_is_admin(f):
    def wrapper(*args, **kwargs): # -> Everything is an [object]
        if kwargs.get('username') != 'admin':
            raise ValueError('User Premission denied!')
        return f(*args, **kwargs)
    return wrapper # -> you return this object

In [97]:
class Store(object):
    
    @check_is_admin
    def get_food(self, username):
        print('You got it')
    
    @check_is_admin
    def put_food(self, username):
        print('You modify it')

In [98]:
s = Store()
s.get_food(username = 'admin')

You got it


#### Tips 1 - equivalent

```python
@check_is_admin <=====> check_is_admin(get_food)
```

#### Tips 2 - no brakets/braces

`func(*args, **kwargs)` means you call the function
```python
class Store(object):
    
    @check_is_admin
    def get_food(self, username):
```

#### Tips 3 - Often used, but not always

#### Tips 4 - Closure
*f is local to check_is_admin*    
*f is nonlocal to wrapper*

In [99]:
def some_function():
    pass

check_is_admin(some_function).__closure__[0]

<cell at 0x000001B9D5FF7490: function object at 0x000001B9D6954670>

In [100]:
@check_is_admin
def some_function():
    pass

some_function.__closure__[0]

<cell at 0x000001B9D5FF7A00: function object at 0x000001B9D6954040>

```python
def check_user_is_not(username):
    def user_check_decorator(f):
        def wrapper(*args, **kwargs):
            if kwargs.get('username') == username:
                raise Exception("This user is not allowed to get food")
            return f(*args, **kwargs)
        return wrapper
    return user_check_decorator

```

#### You are <font color='darkred'>**calling**</font> the outer most function `@check_user_is_not()`
```python
class Store(object):
    @check_user_is_not("admin")
    @check_user_is_not("user123")
    def get_food(self, username, food):
        return self.storage.get(food)
        
```

#### You get a decorator function `user_check_decorator`
```python
def check_user_is_not(username):
    def user_check_decorator(f):
        ............
        ............
        ............
        ............
        ............
    return user_check_decorator

```

#### Pass function f to decorator function, returns `wrapper` function

#### Calling wrapper function,the original function `f` is also called
```python
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
    xxxxxxxxxxxxxxxxxxxxxx:
        def wrapper(*args, **kwargs):
            if kwargs.get('username') == username:
                raise Exception("This user is not allowed to get food")
            return f(*args, **kwargs)
        xxxxxxxxxxxxxxxxxxxxxxxxxxx
    xxxxxxxxxxxxxxxxxxxxxxxxxxx
```


```python
def 一个生产装饰器的工厂(tell_me_what_to_produce):
    def 一个装饰器(我要被装饰):
        def 我就是一个增强的原函数(*args, **kwargs):
            add power to func
            do what func do
            return 我要被装饰(*args, **kwargs)
        return 我就是一个增强的原函数
    return 一个装饰器

```

In [128]:
def 一个生产装饰器的工厂(tell_me_what_to_produce):
    def 一个装饰器(我要被装饰):
        def 我就是一个增强的原函数(*args, **kwargs):
            print('给 `我要被装饰` 增加一点儿能力')
            return 我要被装饰(*args, **kwargs)
        return 我就是一个增强的原函数
    return 一个装饰器